<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.4: Functional Programming
**Prev: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**<br>
**Next: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**

## Motivation
이전의 많은 모듈에서 함수를 보았지만 이제는 직접 만들어 효과적으로 사용할 때입니다.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

이 모듈은 현재 실험 패키지에 있는 Chisel `FixedPoint` 유형을 사용합니다.

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import chisel3.experimental._
import chisel3.internal.firrtl.KnownBinaryPoint

---
# Functional Programming in Scala
스칼라 함수는 모듈 1에서 소개되었으며 이전 모듈에서 많이 사용되는 것을 보았습니다. 
다음은 함수에 대한 리프레시입니다. 
함수는 원하는 수의 입력을 받아 하나의 출력을 생성합니다. 
입력은 종종 함수에 대한 인수(argument)라고 합니다. 
출력을 생성하지 않으려면 `Unit` 유형을 반환합니다.

<span style="color:blue">**Example: Custom Functions**</span><br>
다음은 스칼라의 함수에 대한 몇 가지 예입니다.

In [ ]:
// No inputs or outputs (two versions).
def hello1(): Unit = print("Hello!")
def hello2 = print("Hello again!")

// Math operation: one input and one output.
def times2(x: Int): Int = 2 * x

// Inputs can have default values, and explicitly specifying the return type is optional.
// Note that we recommend specifying the return types to avoid surprises/bugs.
def timesN(x: Int, n: Int = 2) = n * x

// Call the functions listed above.
hello1()
hello2
times2(4)
timesN(4)         // no need to specify n to use the default value
timesN(4, 3)      // argument order is the same as the order where the function was defined
timesN(n=7, x=2)  // arguments may be reordered and assigned to explicitly

## Functions as Objects
스칼라의 함수는 일급 객체입니다. 즉, 함수를 `val`에 할당하고 클래스, 객체 또는 기타 함수에 인수로 전달할 수 있습니다.

<span style="color:blue">**Example: Function Objects**</span><br>
다음은 함수 및 객체로 구현된 동일한 기능입니다.

In [ ]:
// These are normal functions.
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// These are functions as vals.
// The first one explicitly specifies the return type.
val plus1val: Double => Int = x => (x + 1).toInt
val times2val = (x: Int) => x * 2

// Calling both looks the same.
plus1funct(4)
plus1val(4.0)
plus1funct(x=4)
//plus1val(x=4) // this doesn't work

왜 `def` 대신 `val`을 만들고 싶습니까? `val` 을 사용하면 아래와 같이 함수를 다른 함수에 전달할 수 있습니다. 다른 함수를 인수로 받아들이는 고유한 함수를 만들 수도 있습니다. 공식적으로 함수를 취하거나 생성하는 함수를 *고차 함수*라고 합니다. 지난 모듈에서 사용된 것을 보았지만 이제 직접 만들 것입니다!

<span style="color:blue">**Example: Higher-Order Functions**</span><br>
여기서 우리는 다시 `map`을 보여주고, `op`라는 함수를 인수로 받아들이는 새로운 함수 `opN`도 생성합니다.

In [ ]:
def plus1funct(x: Int): Int = x + 1
def times2funct(x: Int): Int = x * 2

// create our function
val plus1 = (x: Int) => x + 1
val times2 = (x: Int) => x * 2

// pass it to map, a list function
val myList = List(1, 2, 5, 9)
val myListPlus = myList.map(plus1)
val myListTimes = myList.map(times2)

// create a custom function, which performs an operation on X N times using recursion
def opN(x: Int, n: Int, op: Int => Int): Int = {
  if (n <= 0) { x }
  else { opN(op(x), n-1, op) }
}

opN(7, 3, plus1)
opN(7, 3, times2)

<span style="color:blue">**Example: Functions vs. Objects**</span><br>
인수 없이 함수를 사용할 때 혼란스러운 상황이 발생할 수 있습니다. 함수는 호출될 때마다 평가되는 반면 `val`은 인스턴스화할 때 평가됩니다.

In [1]:
import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function
val x = Random.nextInt
def y = Random.nextInt

// x was previously evaluated, so it is a constant
println(s"x = $x")
println(s"x = $x")

// y is a function and gets reevaluated at each call, thus these produce different results
println(s"y = $y")
println(s"y = $y")

x = -1276526897
x = -1276526897
y = -417326514
y = -645686402


import scala.util.Random

// both x and y call the nextInt function, but x is evaluated immediately and y is a function

x: Int = -1276526897
defined function y

## Anonymous Functions

이름에서 알 수 있듯이 익명 함수는 이름이 없습니다. 한 번만 사용할 경우 함수에 대해 `val`을 만들 필요가 없습니다.

<span style="color:blue">**Example: Anonymous Functions**</span><br>
다음 예제는 이를 보여줍니다. 그것들은 종종 범위가 지정됩니다(괄호 대신 중괄호 안에 넣음).

In [3]:
val myList = List(5, 6, 7, 8)

// add one to every item in the list using an anonymous function
// arguments get passed to the underscore variable
// these all do the same thing
myList.map( (x:Int) => x + 1 )
myList.map(_ + 1)

// a common situation is to use case statements within an anonymous function
val myAnyList = List(1, 2, "3", 4L, myList)
myAnyList.map {
  case (_:Int|_:Long) => "Number"
  case _:String => "String"
  case _ => "error"
}

myList: List[Int] = List(5, 6, 7, 8)
res2_1: List[Int] = List(6, 7, 8, 9)
res2_2: List[Int] = List(6, 7, 8, 9)
myAnyList: List[Any] = List(1, 2, "3", 4L, List(5, 6, 7, 8))
res2_4: List[String] = List("Number", "Number", "String", "Number", "error")

<span style="color:red">**Exercise: Sequence Manipulation**</span><br>
사용하게 될 고차 함수의 일반적인 집합은 `scanLeft`/`scanRight`, `reduceLeft`/`reduceRight` 및 `foldLeft`/`foldRight`입니다. 각각이 어떻게 작동하고 언제 사용해야 하는지 이해하는 것이 중요합니다. 'scan', 'reduce', 'fold'에 대한 기본 방향이 남아 있지만 이것이 모든 경우에 보장되는 것은 아닙니다.

In [ ]:
val exList = List(1, 5, 7, 100)

// write a custom function to add two numbers, then use reduce to find the sum of all values in exList
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

// find the sum of exList using an anonymous function (hint: you've seen this before!)
val anon_sum = exList.reduce(_ + _)

// find the moving average of exList from right to left using scan; make the result (ma2) a list of doubles
def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)

In [10]:
assert(add(88, 88) == 176)
assert(sum == 113)

assert(anon_sum == 113)

assert(avg(100, 100.0) == 100.0)
assert(ma2 == List(8.875, 16.75, 28.5, 50.0, 0.0))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def add(a: Int, b: Int): Int = a + b
val sum = exList.reduce(add)

val anon\_sum = exList.reduce(\_ + \_)

def avg(a: Int, b: Double): Double = (a + b)/2.0
val ma2 = exList.scanRight(0.0)(avg)
</pre></article></div></section></div>

---
# Functional Programming in Chisel
Chisel에서 하드웨어 생성기를 생성할 때 함수형 프로그래밍을 사용하는 방법에 대한 몇 가지 예를 살펴보겠습니다.

<span style="color:blue">**Example: FIR Filter**</span><br>
먼저 이전 예제의 FIR 필터를 다시 살펴보겠습니다. 
coefficient를 클래스에 매개변수로 전달하거나 프로그래밍 가능하게 만드는 대신 
window coefficient를 계산하는 방법을 정의하는 함수를 FIR에 전달합니다. 
이 함수는 window 길이와 비트 너비를 사용하여 coefficient를의 크기 조정된 list를 생성합니다. 
다음은 두 개의 예제 창입니다. 
분수를 피하기 위해 최대 및 최소 정수 값 사이가 되도록 계수를 조정합니다. 
이 창에 대한 자세한 내용은 [이 Wikipedia 페이지](https://en.wikipedia.org/wiki/Window_function)를 확인하세요.

In [7]:
// get some math functions
import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window
val TriangularWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x:Int) => 1-abs((x.toDouble-(length-1)/2.0)/((length-1)/2.0)) )
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

// Hamming window
val HammingWindow: (Int, Int) => Seq[Int] = (length, bitwidth) => {
  val raw_coeffs = (0 until length).map( (x: Int) => 0.54 - 0.46*cos(2*Pi*x/(length-1)))
  val scaled_coeffs = raw_coeffs.map( (x: Double) => round(x * pow(2, bitwidth)).toInt)
  scaled_coeffs
}

val a = 0 until 3

print(a)

// check it out! first argument is the window length, and second argument is the bitwidth
TriangularWindow(10, 16)
HammingWindow(10, 16)

Range 0 until 3

import scala.math.{abs, round, cos, Pi, pow}

// simple triangular window

TriangularWindow: (Int, Int) => Seq[Int] = ammonite.$sess.cmd6$Helper$$Lambda$3314/1941899394@4bf0aa04
HammingWindow: (Int, Int) => Seq[Int] = ammonite.$sess.cmd6$Helper$$Lambda$3315/465003634@f6a18c0
a: Range = Range(0, 1, 2)
res6_5: Seq[Int] = Vector(
  0,
  14564,
  29127,
  43691,
  58254,
  58254,
  43691,
  29127,
  14564,
  0
)
res6_6: Seq[Int] = Vector(
  5243,
  12296,
  30155,
  50463,
  63718,
  63718,
  50463,
  30155,
  12296,
  5243
)

이제 창 함수를 인수로 받아들이는 FIR 필터를 만듭니다. 이를 통해 나중에 새 창을 정의하고 동일한 FIR 생성기를 유지할 수 있습니다. 또한 윈도우가 다른 길이나 비트폭에 대해 다시 계산될 것임을 알고 FIR의 크기를 독립적으로 조정할 수 있습니다. 컴파일 타임에 창을 선택하기 때문에 이러한 계수는 고정되어 있습니다.

In [0]:
// our FIR has parameterized window length, IO bitwidth, and windowing function
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy
  })

  // calculate the coefficients using the provided window function, mapping to UInts
  val coeffs = window(length, bitwidth).map(_.U)
  
  // create an array holding the output of the delays
  // note: we avoid using a Vec here since we don't need dynamic indexing
  val delays = Seq.fill(length)(Wire(UInt(bitwidth.W))).scan(io.in)( (prev: UInt, next: UInt) => {
    next := RegNext(prev)
    next
  })
  
  // multiply, putting result in "mults"
  val mults = delays.zip(coeffs).map{ case(delay: UInt, coeff: UInt) => delay * coeff }
  
  // add up multiplier outputs with bit growth
  val result = mults.reduce(_+&_)

  // connect output
  io.out := result
}

visualize(() => new MyFir(5, 12, TriangularWindow))

cmd0.sc:27: not found: value visualize
val res0_1 = visualize(() => new MyFir(5, 12, TriangularWindow))
             ^cmd0.sc:1: not found: type Module
class MyFir(length: Int, bitwidth: Int, window: (Int, Int) => Seq[Int]) extends Module {
                                                                                ^cmd0.sc:27: not found: value TriangularWindow
val res0_1 = visualize(() => new MyFir(5, 12, TriangularWindow))
                                              ^cmd0.sc:2: not found: value IO
  val io = IO(new Bundle {
           ^cmd0.sc:2: not found: type Bundle
  val io = IO(new Bundle {
                  ^cmd0.sc:3: not found: value UInt
    val in = Input(UInt(bitwidth.W))
                   ^cmd0.sc:3: value W is not a member of Int
    val in = Input(UInt(bitwidth.W))
                                 ^cmd0.sc:4: not found: value Output
    val out = Output(UInt((bitwidth*2+length-1).W)) // expect bit growth, conservative but lazy
              ^cmd0.sc:4: not found:

: 

마지막 세 줄은 쉽게 하나로 결합될 수 있습니다. 또한 손실을 피하기 위해 비트폭 증가를 보수적으로 처리한 방법에 주목하십시오.

<span style="color:blue">**Example: FIR Filter Tester**</span><br>
FIR을 테스트해봅시다! 이전에는 사용자 지정 골든 모델을 제공했습니다. 이번에는 유용한 선형 대수 및 신호 처리 함수의 Scala 라이브러리인 Breeze를 FIR 필터의 황금 모델로 사용할 것입니다. 아래 코드는 Chisel 출력을 골든 모델 출력과 비교하며 오류가 있으면 테스터가 실패합니다.

예상 호출 직후 끝에 있는 인쇄 문에 대한 주석 처리를 제거해 보십시오. 또한 창을 삼각형에서 해밍으로 변경해 보십시오.

In [43]:
// math imports
import scala.math.{pow, sin, Pi}
import breeze.signal.{filter, OptOverhang}
import breeze.signal.support.{CanFilter, FIRKernel1D}
import breeze.linalg.DenseVector

// test parameters
val length = 7
val bitwidth = 12 // must be less than 15, otherwise Int can't represent the data, need BigInt
val window = TriangularWindow

// test our FIR
test(new MyFir(length, bitwidth, window)) { c =>
    
    // test data
    val n = 100 // input length
    val sine_freq = 10
    val samp_freq = 100

    // sample data, scale to between 0 and 2^bitwidth
    val max_value = pow(2, bitwidth)-1
    val sine = (0 until n).map(i => (max_value/2 + max_value/2*sin(2*Pi*sine_freq/samp_freq*i)).toInt)
    //println(s"input = ${sine.toArray.deep.mkString(", ")}")

    // coefficients
    val coeffs = window(length, bitwidth)
    //println(s"coeffs = ${coeffs.toArray.deep.mkString(", ")}")

    // use breeze filter as golden model; need to reverse coefficients
    val expected = filter(
        DenseVector(sine.toArray),
        FIRKernel1D(DenseVector(coeffs.reverse.toArray), 1.0, ""),
        OptOverhang.None
    )
    expected.toArray // seems to be necessary
    //println(s"exp_out = ${expected.toArray.deep.mkString(", ")}") // this seems to be necessary

    // push data through our FIR and check the result
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- 0 until n) {
        c.io.in.poke(sine(i).U)
        if (i >= length-1) { // wait for all registers to be initialized since we didn't zero-pad the data
            val expectValue = expected(i-length+1)
            //println(s"expected value is $expectValue")
            c.io.out.expect(expected(i-length+1).U)
            //println(s"cycle $i, got ${c.io.out.peek()}, expect ${expected(i-length+1)}")
        }
        c.clock.step(1)
    }
}

test MyFir Success: 0 tests passed in 107 cycles in 0.222556 seconds 480.78 Hz


import scala.math.{pow, sin, Pi}

import breeze.signal.{filter, OptOverhang}

import breeze.signal.support.{CanFilter, FIRKernel1D}

import breeze.linalg.DenseVector

// test parameters

length: Int = 7
bitwidth: Int = 12
window: (Int, Int) => Seq[Int] = ammonite.$sess.cmd40$Helper$$Lambda$3748/663073470@70f2fb1c

---
# Chisel Exercises
다음 연습을 완료하여 함수 작성, 하드웨어 생성기에 대한 인수로 사용, 변경 가능한 데이터 피하기를 연습하십시오.

<span style="color:red">**Exercise: Neural Network Neuron**</span><br>
첫 번째 예에서는 인공 신경망에서 완전히 연결된 계층의 빌딩 블록인 뉴런을 구축하게 됩니다. 뉴런은 입력당 하나씩 입력과 가중치 집합을 가져와 하나의 출력을 생성합니다. 가중치와 입력을 곱하고 더하고 그 결과는 활성화 함수를 통해 제공됩니다. 이 연습에서는 다양한 활성화 함수를 구현하고 이를 뉴런 생성기에 인수로 전달합니다.

![Neuron](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/ArtificialNeuronModel_english.png/600px-ArtificialNeuronModel_english.png)

먼저 다음 코드를 완성하여 뉴런 생성기를 만듭니다. 매개변수 `inputs`는 입력 수를 제공합니다. 'act' 매개변수는 활성화 함수의 논리를 구현하는 함수입니다. 8개의 소수 비트를 사용하여 입력 및 출력을 16비트 고정 소수점 값으로 만들 것입니다.

In [12]:
class Neuron(inputs: Int, act: FixedPoint => FixedPoint) extends Module {
  val io = IO(new Bundle {
    val in      = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val weights = Input(Vec(inputs, FixedPoint(16.W, 8.BP)))
    val out     = Output(FixedPoint(16.W, 8.BP))
  })
  
  ???
}

defined class Neuron

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val mac = io.in.zip(io.weights).map{ case(a:FixedPoint, b:FixedPoint) => a*b}.reduce(_+_)
  io.out := act(mac)
</pre></article></div></section></div>

이제 활성화 함수를 생성해 보겠습니다! 임계값 0을 사용합니다. 일반적인 활성화 함수는 시그모이드 함수와 정류 선형 단위(ReLU)입니다.

우리가 사용할 시그모이드를 [로지스틱 함수](https://en.wikipedia.org/wiki/Logistic_function)라고 하며,

$logistic(x) = \cfrac{1}{1+e^{-\beta x}}$

여기서 $\beta$는 기울기 계수입니다. 그러나 하드웨어에서 지수 함수를 계산하는 것은 상당히 어렵고 비용이 많이 듭니다. 우리는 이것을 단계 함수로 근사할 것입니다.
$step(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             1  & \text{if } x \gt 0
       \end{cases}$

두 번째 함수인 ReLU는 유사한 공식으로 제공됩니다.

$relu(x) = \begin{cases}
             0  & \text{if } x \le 0 \\
             x  & \text{if } x \gt 0
       \end{cases}$

아래 두 가지 기능을 구현하십시오. `-3.14.F(8.BP)`와 같은 고정 소수점 리터럴을 지정할 수 있습니다.

In [13]:
val Step: FixedPoint => FixedPoint = ???
val ReLU: FixedPoint => FixedPoint = ???

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val Step: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), 1.F(8.BP))
val ReLU: FixedPoint => FixedPoint = x => Mux(x <= 0.F(8.BP), 0.F(8.BP), x)
</pre></article></div></section></div>

마지막으로 뉴런의 정확성을 확인하는 테스터를 만들어 보겠습니다. 단계 활성화 기능을 사용하면 뉴런을 논리 게이트 근사기로 사용할 수 있습니다. 가중치와 편향을 적절히 선택하면 이진 기능을 수행할 수 있습니다. AND 논리를 사용하여 뉴런을 테스트합니다. 다음 테스터를 완료하여 단계 함수로 뉴런을 확인하십시오.

회로는 순전히 조합이므로 `reset(5)` 및 `step(1)` 호출이 필요하지 않습니다.

In [13]:
// test our Neuron 
test(new Neuron(2, Step)) { c =>
    val inputs = Seq(Seq(-1, -1), Seq(-1, 1), Seq(1, -1), Seq(1, 1))

    // make this a sequence of two values
    val weights = ???

    // push data through our Neuron and check the result (AND gate)
    c.reset.poke(true.B)
    c.clock.step(5)
    c.reset.poke(false.B)
    for (i <- inputs) {
        c.io.in(0).poke(i(0).F(8.BP))
        c.io.in(1).poke(i(1).F(8.BP))
        c.io.weights(0).poke(weights(0).F(16.W, 8.BP))
        c.io.weights(1).poke(weights(1).F(16.W, 8.BP))
        c.io.out.expect((if (i(0) + i(1) > 0) 1 else 0).F(16.W, 8.BP))
        c.clock.step(1)
    }
    
}

cmd13.sc:1: not found: value Step
val res13 = test(new Neuron(2, Step)) { c =>
                               ^Compilation Failed

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
val weights  = Seq(1.0, 1.0)
</pre></article></div></section></div>

---
# You're done!

[Return to the top.](#top)